In [1]:
import copy
import numpy as np
from sklearn import metrics

import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.datasets import HeterophilousGraphDataset

# Load dataset.
dataset = HeterophilousGraphDataset(root="./data", name="Roman-Empire")
data = dataset[0]


In [2]:
# Define GPR-GNN model.

class GPRGNNConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr='add')

    def forward(self, x, edge_index):
        # Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Compute normalization for symmetric normalized matrix.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Propagate messages.
        out = self.propagate(edge_index, x=x, norm=norm)

        return out

    def message(self, x_j, norm):
        # Normalize neighboring vectors.
        return norm.view(-1, 1) * x_j


class GPRGNN(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        hidden_channels: int,
        K: int,
        nn_dropout_p: float,
        gpr_dropout_p: float
    ):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Dropout(p=nn_dropout_p),
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(p=nn_dropout_p),
            nn.Linear(hidden_channels, out_channels)
        )

        self.conv = nn.ModuleList([
            GPRGNNConv() for _ in range(K)
        ])

        self.gpr_dropout = nn.Dropout(p=gpr_dropout_p)

        self.gamma = nn.Parameter(torch.rand(size=(K + 1,)))

    def forward(self, x, edge_index):
        # Project initial features with a MLP.
        x = self.mlp(x)

        # Dropout to prevent overfitting.
        x = self.gpr_dropout(x)

        # Combine hidden representations with GPR weights.
        z = self.gamma[0] * x
        for i in range(K):
            x = self.conv[i](x=x, edge_index=edge_index)
            z = z + self.gamma[i + 1] * x

        return z


In [3]:
# Hyperparameters are set according to validation accuracy and my intuition, where starting
# values were taken from papers: "Adaptive Universal Generalized PageRank Graph Neural Network"
# and "A critical look at the evaluation of GNNs under heterophily: are we really making progress?".

hidden_dim = 512        # hidden dimension for MLP
K = 5                   # degree of polynomial in GPR
nn_dropout_p = 0.5      # dropout for MLP
gpr_dropout_p = 0.5     # dropout before calculation of GPR score

num_experiments = 10
num_epochs = 100
learning_rate = 1e-2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()


def train(model, data, experiment_ind: int):
    model.train()
    optimizer.zero_grad()

    train_mask = data.train_mask[:, experiment_ind]

    logits = model(x=data.x, edge_index=data.edge_index)[train_mask]
    loss = criterion(logits, data.y[train_mask])
    loss.backward()

    optimizer.step()


@torch.no_grad()
def test(model, data, test_set: bool, experiment_ind: int):
    model.eval()
    logits, accs = model(x=data.x, edge_index=data.edge_index), []

    data_splits = ["train_mask", "val_mask"]
    if test_set:
        data_splits.append("test_mask")

    for mask_name in data_splits:
        mask = getattr(data, mask_name)
        mask = mask[:, experiment_ind]

        pred = logits[mask].max(dim=1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)

    return accs


train_accs, val_accs, test_accs = [], [], []
for experiment_ind in range(num_experiments):
    # Reinitialize the model for each experiment.
    model = GPRGNN(
        in_channels=dataset.num_features,
        out_channels=dataset.num_classes,
        hidden_channels=hidden_dim,
        K=K,
        nn_dropout_p=nn_dropout_p,
        gpr_dropout_p=gpr_dropout_p,
    )

    model, data = model.to(device), data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model.
    for epoch in range(0, num_epochs):
        train(model, data, experiment_ind=experiment_ind)
        if epoch % 10 == 0:
            train_acc, val_acc = test(model, data, test_set=False, experiment_ind=experiment_ind)
            print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}')
    
    # Evaluate the model on test set.
    train_acc, val_acc, test_acc = test(model, data, test_set=True, experiment_ind=experiment_ind)
    print(f'FINAL PERFORMANCE OF EXPERIMENT {experiment_ind}: Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    test_accs.append(test_acc)

Epoch: 000, Train: 0.2137, Val: 0.2104
Epoch: 010, Train: 0.5041, Val: 0.4883
Epoch: 020, Train: 0.5850, Val: 0.5626
Epoch: 030, Train: 0.6495, Val: 0.6233
Epoch: 040, Train: 0.6771, Val: 0.6501
Epoch: 050, Train: 0.6942, Val: 0.6579
Epoch: 060, Train: 0.7069, Val: 0.6669
Epoch: 070, Train: 0.7137, Val: 0.6688
Epoch: 080, Train: 0.7222, Val: 0.6757
Epoch: 090, Train: 0.7312, Val: 0.6789
FINAL PERFORMANCE OF EXPERIMENT 0: Train: 0.7347, Val: 0.6796, Test: 0.6784
Epoch: 000, Train: 0.1679, Val: 0.1656
Epoch: 010, Train: 0.4796, Val: 0.4722
Epoch: 020, Train: 0.5631, Val: 0.5518
Epoch: 030, Train: 0.6321, Val: 0.6199
Epoch: 040, Train: 0.6679, Val: 0.6489
Epoch: 050, Train: 0.6935, Val: 0.6609
Epoch: 060, Train: 0.7091, Val: 0.6720
Epoch: 070, Train: 0.7166, Val: 0.6763
Epoch: 080, Train: 0.7220, Val: 0.6837
Epoch: 090, Train: 0.7292, Val: 0.6824
FINAL PERFORMANCE OF EXPERIMENT 1: Train: 0.7311, Val: 0.6810, Test: 0.6867
Epoch: 000, Train: 0.1549, Val: 0.1511
Epoch: 010, Train: 0.4525, Va

In [4]:
# Calculate overall performance.
train_accs = np.array(train_accs)
val_accs = np.array(val_accs)
test_accs = np.array(test_accs)

print(f'OVERALL PERFORMANCE')
print(f'Train: {np.mean(train_accs):.4f} ± {np.std(train_accs):.4f}')
print(f'Val: {np.mean(val_accs):.4f} ± {np.std(val_accs):.4f}')
print(f'Test: {np.mean(test_accs):.4f} ± {np.std(test_accs):.4f}')

OVERALL PERFORMANCE
Train: 0.7336 ± 0.0037
Val: 0.6753 ± 0.0048
Test: 0.6770 ± 0.0069


In [5]:
@torch.no_grad()
def uncertainty_estimation(model, data):
    model.eval()
    logits = model(x=data.x, edge_index=data.edge_index)
    preds = nn.functional.softmax(logits, dim=-1)
    
    # Calculate entropies.
    log_preds = torch.log2(preds)
    log_preds[log_preds == float('-inf')] = 0
    entropies = -torch.sum(preds * log_preds, dim=-1)

    assert torch.all(entropies >= 0)    

    test_entropies = entropies[data.test_mask]
    ood_test_examples = data.ood_test_mask[data.test_mask].int()

    auc = metrics.roc_auc_score(y_true=ood_test_examples, y_score=test_entropies)
    return auc

### Out-of-distribution  examples with features sampled from the standard normal distribution

In [6]:
# Create dataset with out-of-distribution examples.
perturbed_data = copy.deepcopy(data)

# Let's limit to only a single experiment since we want to perturb only features within test set.
# Take the last experiment since we already have a trained model for that split.
experiment_ind = 9
perturbed_data.test_mask = perturbed_data.test_mask[:, experiment_ind]
num_nodes, num_features = perturbed_data.x.shape

# Choose approximately half of test nodes as out-of-distribution examples.
ood_mask = torch.rand(size=(perturbed_data.x.shape[0],), device=perturbed_data.test_mask.device) < 0.5
perturbed_data.ood_test_mask = perturbed_data.test_mask & ood_mask

# Randomly sample features from the standard normal distribution for out-of-distribution examples.
num_ood_test_examples, num_features = torch.sum(perturbed_data.ood_test_mask), perturbed_data.x.shape[1]
perturbed_data.x[perturbed_data.ood_test_mask] = torch.normal(size=(num_ood_test_examples, num_features), mean=0, std=1)

In [7]:
uncertainty_auc = uncertainty_estimation(model, perturbed_data)
print(f"UNCERTAINTY AUC SCORE FOR EXPERIMENT {experiment_ind}: {uncertainty_auc}")

UNCERTAINTY AUC SCORE FOR EXPERIMENT 9: 0.10524300681624174


### Out-of-distribution examples with features sampled from normal distribution with the original mean and std

In [8]:
# Create dataset with out-of-distribution examples.
perturbed_data = copy.deepcopy(data)

# Let's limit to only a single experiment since we want to perturb only features within test set.
# Take the last experiment since we already have a trained model for that split.
experiment_ind = 9
perturbed_data.test_mask = perturbed_data.test_mask[:, experiment_ind]
num_nodes, num_features = perturbed_data.x.shape

# Choose approximately half of test nodes as out-of-distribution examples.
ood_mask = torch.rand(size=(perturbed_data.x.shape[0],), device=perturbed_data.test_mask.device) < 0.5
perturbed_data.ood_test_mask = perturbed_data.test_mask & ood_mask

# Randomly sample features from the normal distribution with the original mean and std for out-of-distribution examples.
num_ood_test_examples, num_features = torch.sum(perturbed_data.ood_test_mask), perturbed_data.x.shape[1]
mean = torch.mean(perturbed_data.x, dim=0).reshape(1, -1)
std = torch.std(perturbed_data.x, dim=0).reshape(1, -1)
perturbed_data.x[perturbed_data.ood_test_mask] = torch.normal(mean=mean.repeat(num_ood_test_examples, 1), std=std.repeat(num_ood_test_examples, 1))

In [9]:
uncertainty_auc = uncertainty_estimation(model, perturbed_data)
print(f"UNCERTAINTY AUC SCORE FOR EXPERIMENT {experiment_ind}: {uncertainty_auc}")

UNCERTAINTY AUC SCORE FOR EXPERIMENT 9: 0.8475519976914421
